In [61]:
import pandas as pd
import sqlite3

In [65]:
# Charger le CSV
df = pd.read_csv("data/24-25.csv")

# Créer une base SQLite (ici en mémoire)
conn = sqlite3.connect(":memory:")

# Importer le DataFrame dans SQLite comme une table appelée "L1"
df.to_sql("L1", conn, index=False, if_exists="replace")

306

In [66]:
query = """
SELECT * FROM L1
"""
result = pd.read_sql(query, conn)
print(result)

    Div        Date   Time    HomeTeam     AwayTeam  FTHG  FTAG FTR  HTHG  \
0    F1  16/08/2024  19:45    Le Havre     Paris SG     1     4   A     0   
1    F1  17/08/2024  16:00       Brest    Marseille     1     5   A     1   
2    F1  17/08/2024  18:00       Reims        Lille     0     2   A     0   
3    F1  17/08/2024  20:00      Monaco   St Etienne     1     0   H     1   
4    F1  18/08/2024  14:00     Auxerre         Nice     2     1   H     1   
..   ..         ...    ...         ...          ...   ...   ...  ..   ...   
301  F1  17/05/2025  20:00      Nantes  Montpellier     3     0   H     2   
302  F1  17/05/2025  20:00        Nice        Brest     6     0   H     3   
303  F1  17/05/2025  20:00    Paris SG      Auxerre     3     1   H     0   
304  F1  17/05/2025  20:00  St Etienne     Toulouse     2     3   A     1   
305  F1  17/05/2025  20:00  Strasbourg     Le Havre     2     3   A     1   

     HTAG  ... B365CAHH  B365CAHA  PCAHH  PCAHA  MaxCAHH  MaxCAHA  AvgCAHH 

# 1. Analyses sportives "classiques"

## Classement des équipes

- total de points (3 victoire, 1 nul, 0 défaite), différence de buts (FTHG - FTAG), trié comme un vrai classement de Ligue 1

In [71]:
query = """
SELECT Team, SUM(BP) AS BP, SUM(BC) AS BC, SUM(BP) - SUM(BC) AS DB, SUM(Points) AS Points
FROM (
        -- Equipe Domicile
        SELECT HomeTeam AS Team, FTHG AS BP, FTAG AS BC,
        CASE
            WHEN FTR = 'H' THEN 3
            WHEN FTR = 'A' THEN 0
            WHEN FTR = 'D' THEN 1
        END AS Points
        FROM L1

        UNION ALL

        -- Equipe Extérieur
        SELECT AwayTeam AS Team, FTAG AS BP, FTHG AS BC,
        CASE
            WHEN FTR = 'H' THEN 0
            WHEN FTR = 'A' THEN 3
            WHEN FTR = 'D' THEN 1
        END AS Points
        FROM L1

) AS Matchs
GROUP BY Team
ORDER BY Points DESC, DB DESC
"""

result = pd.read_sql(query, conn)
print(result)

           Team  BP  BC  DB  Points
0      Paris SG  92  35  57      84
1     Marseille  74  47  27      65
2        Monaco  63  41  22      61
3          Nice  66  41  25      60
4         Lille  52  36  16      60
5          Lyon  65  46  19      57
6    Strasbourg  56  44  12      57
7          Lens  42  39   3      52
8         Brest  52  59  -7      50
9      Toulouse  44  43   1      42
10      Auxerre  48  51  -3      42
11       Rennes  51  50   1      41
12       Nantes  39  52 -13      36
13       Angers  32  53 -21      36
14     Le Havre  40  71 -31      34
15        Reims  33  47 -14      33
16   St Etienne  39  77 -38      30
17  Montpellier  23  79 -56      16


## Performances domicile / extérieur

- comparer points, buts marqués et encaissés à domicile vs à l’extérieur

In [102]:
query = """
SELECT H.Team, H.BPHome, H.BCHome, H.DBHome, H.HomePoints,
        A.BPAway, A.BCAway, A.DBAway, A.AwayPoints
FROM (
        SELECT HomeTeam AS Team, SUM(FTHG) AS BPHome, SUM(FTAG) AS BCHome, SUM(FTHG) - SUM(FTAG) AS DBHome,
        SUM(CASE
            WHEN FTR = 'H' THEN 3
            WHEN FTR = 'A' THEN 0
            WHEN FTR = 'D' THEN 1
        END) AS HomePoints
        FROM L1
        GROUP BY Team
        ORDER BY HomePoints DESC
) AS H
JOIN (
        SELECT AwayTeam AS Team, SUM(FTAG) AS BPAway, SUM(FTHG) AS BCAway, SUM(FTAG) - SUM(FTHG) AS DBAway,
        SUM(CASE
            WHEN FTR = 'H' THEN 0
            WHEN FTR = 'A' THEN 3
            WHEN FTR = 'D' THEN 1
        END) AS AwayPoints
        FROM L1
        GROUP BY Team
        ORDER BY AwayPoints DESC
) AS A
ON H.Team = A.Team
ORDER BY (H.HomePoints + A.AwayPoints) DESC
"""
result = pd.read_sql(query, conn)
print(result)

           Team  BPHome  BCHome  DBHome  HomePoints  BPAway  BCAway  DBAway  \
0      Paris SG      45      16      29          44      47      19      28   
1     Marseille      41      23      18          34      33      24       9   
2        Monaco      38      16      22          39      25      25       0   
3          Nice      38      15      23          37      28      26       2   
4         Lille      31      18      13          37      21      18       3   
5    Strasbourg      33      20      13          35      23      24      -1   
6          Lyon      37      21      16          35      28      25       3   
7          Lens      19      22      -3          23      23      17       6   
8         Brest      31      21      10          33      21      38     -17   
9       Auxerre      24      17       7          27      24      34     -10   
10     Toulouse      20      22      -2          20      24      21       3   
11       Rennes      25      17       8          28 

## Buts par période

- comparer HTHG / HTAG (mi-temps) vs FTHG / FTAG (fin de match) → voir quelles équipes marquent plutôt en 2ème mi-temps

## Attaque et défense

- équipe qui tire le plus (HS + AS), équipe qui cadre le plus (HST + AST), équipe qui encaisse le plus de cartons (HY, AY, HR, AR)

# 2. Analyses "data métier"

## Corrélation tirs / buts

- moyenne de tirs cadrés vs buts marqués, pour identifier les équipes les plus efficaces

## Fair-play / agressivité

- ratio fautes (HF, AF) et cartons (HY, AY) par match → voir si une équipe est plus "rugueuse"

## Possibilité d’upsets (surprises)

- comparer la cote du favori (B365H, B365A) avec le résultat effectif (FTR)

## Analyse de la variance des cotes

- comparer MaxH / MinH pour voir les matchs où les bookmakers n’étaient pas d’accord

# 3. Analyses "pari sportif"

## Taux de victoire attendu vs réel

- transformer une cote (1 / cote) en probabilité implicite, comparer avec la fréquence réelle des victoires/nuls

## Valeur des paris "over/under 2.5 buts"

- comparer la prédiction des bookmakers avec le nombre réel de matchs > 2.5 buts

## Analyse des handicaps asiatiques

- voir si les résultats réels sont cohérents avec les handicaps

## Simulation de stratégie

- appliquer une stratégie simple (ex. toujours parier sur le favori à domicile avec cote < 1.5) et calculer le ROI